# 4.2.0 Attention
# Key and Query
A A B A _ (B)
Keys: embeddings of sequence
Query: what we want to find

Assume A = [1 0] B=[0 1] (one-hot encoding)
call them ei

Step 1: Compute score for each key given query

[1 0] [1 0] [0 1] [1 0] -> keys
A     A     B     A

score si = ki^T * q (taking the ith key and dotting it with the query)
set query q = [0 1] find Bs

Step 2: softmax
A A B A
0 0 1 0 => softmax: 1/6 1/6 1/2 1/6 (assume e=3) alpha-i

Step 3: Compute output as a weighted sum of the input
result: sum(i=1 to n)alpha-1*ei = (1/6[1 0]+1/6[1 0]+1/2[0 1]+1/6[1 0])
        = [1/2 1/2]
        compare to average: [3/4 1/4]

we can make attention more peaked by amplifying the embeddings
ki = Wk*ei  
Wk= diagonal matrix with 10s on the diagonal  Wk = 10 0 
                                                    0 10
[10,0] [10,0] [0,10] [10,0]
0      0      1      0
what will new attention values be with these keys?
[1/(e^10+3) 1/(e^10+3) e^10/(e^10+3) 1/(e^10+3)]

- Original dot product attention: si = ki^T q
- Scaled dot product attention: si = ki^T Wq
- Equivalent to having two weight matrices: si = (Wk ki)^T(Wq q)



# 4.2.1 Self-attention
self-attention: builds on the idea of attention. Every word in a sequence is both a key and a query simultaneously

Q: seq len x d matrix (d = embedding dimension = 2 for these slides)
K: seq len x d matrix
Wq = 0 1
     0 1 no matter what the value is, we're going to look for Bs
Wk = 10 0
     0  10 "booster" as before

E = (1 0     Wq = 0 1       Wk = 10 0
    1 0           0 1             0 10
    0 1
    1 0)

Q = E Wq = (0 1
            0 1
            0 1
            0 1)  -> every token looking for if any B around
K = E Wk = (10 0
            10 0
            0 10
            10 0)
scores: S = Q K^T
        Sij = qi dot kj
        S = (0 0 10 0
            0 0 10 0
            0 0 10 0
            0 0 10 0) - rows represent attention scores
e.g. row 1: "I care about row 3"
-> row-wise softmax (A) => distribution per row

output = AE (actually A(E W^v))

Attention(Q,K,V) = softmax(Q * K^T / sqrt(dk)) V = Z
Q = EWq, K=EWk, V=EWv
where dk = dimension of the keys / vector dimension - control the scale, less peaked
this is just one head of self-attention - produce multiple heads via randomly initialize parameter matrices

What does self-attention produce?
- Square attention matrix * input = same dimension as the input
- Computes a contextualized encoding for each word, preserving the length of the sequence

# Properties of self-attention
complexity per layer: O(n^2 * d)
sequential operation: O(1)
max path length: O(1)
n = sentence length, d = hidden dim, k = kernel size, r = restricted neighborhood size
- Quadratic complexity, but O(1) sequential operations (not linear like in RNNs) and
    O(1) "path" for words to inform each other


![Image](4.2.1_1.png)

![](4.2.1_2.png)

# Exercise
What is the complexity of computing the self-attention matrix for a sentence with 
 words? Assume the input embeddings are size k_1, the key and value embeddings are size k_2, and we're doing naive matrix multiplication.

1. Linear projection - X*W_q/h/v -> (n*k_1)*(k_1*k_2) -> O(n*k_1*k_2)
2. Attention score - QK^T -> (n*k_2)*(k_2*n) -> O(n^2 * k_2)
3. Softmax - Just elementwise operations on an (𝑛×𝑛) matrix → O(n ^2)
4. Weighted sum - Multiply (n×n) attention scores with V(n×k_2) -> O(n^2*k_2)
